# Analysis of Churn Data of a Telecom company #

**by Alexander Abstreiter**

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Exploration" data-toc-modified-id="Exploration-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exploration</a></span></li><li><span><a href="#Principal-component-analysis" data-toc-modified-id="Principal-component-analysis-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Principal component analysis</a></span></li><li><span><a href="#Comparison-of-differently-preprocessed-datasets-for-classification" data-toc-modified-id="Comparison-of-differently-preprocessed-datasets-for-classification-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Comparison of differently preprocessed datasets for classification</a></span><ul class="toc-item"><li><span><a href="#Without-additional-balancing-techniques" data-toc-modified-id="Without-additional-balancing-techniques-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Without additional balancing techniques</a></span></li><li><span><a href="#Using-class-weights-in-the-loss-function" data-toc-modified-id="Using-class-weights-in-the-loss-function-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Using class weights in the loss function</a></span></li><li><span><a href="#Using-Synthetic-Minority-Over-sampling-Technique" data-toc-modified-id="Using-Synthetic-Minority-Over-sampling-Technique-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>Using Synthetic Minority Over-sampling Technique</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Comparison</a></span></li></ul></li><li><span><a href="#Classification" data-toc-modified-id="Classification-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Classification</a></span><ul class="toc-item"><li><span><a href="#Logistic-Regression" data-toc-modified-id="Logistic-Regression-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Logistic Regression</a></span></li><li><span><a href="#K-Nearest-Neighbors" data-toc-modified-id="K-Nearest-Neighbors-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>K-Nearest Neighbors</a></span></li><li><span><a href="#Support-Vector-Machine" data-toc-modified-id="Support-Vector-Machine-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Support-Vector Machine</a></span></li><li><span><a href="#Decision-Tree" data-toc-modified-id="Decision-Tree-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Decision Tree</a></span></li><li><span><a href="#Random-Forest" data-toc-modified-id="Random-Forest-5.5"><span class="toc-item-num">5.5&nbsp;&nbsp;</span>Random Forest</a></span></li><li><span><a href="#Comparison" data-toc-modified-id="Comparison-5.6"><span class="toc-item-num">5.6&nbsp;&nbsp;</span>Comparison</a></span></li></ul></li><li><span><a href="#Conclusion" data-toc-modified-id="Conclusion-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Conclusion</a></span></li></ul></div>

## Introduction ##

In this report, a dataset on churn data of a Telecom company is analysed. It can be found here: https://www.kaggle.com/becksddf/churn-in-telecoms-dataset.

The dataset contains data on the customers of a Telecom company.
Each row represents a customer and the columns contain customer’s attributes which are described in the following:
- state: the state the user lives in
- account length: the number of days the user has this account
- area code: the code of the area the user lives in
- phone number: the phone number of the user
- international plan: true if the user has the international plan, otherwise false
- voice mail plan: true if the user has the voice mail plan, otherwise false
- number vmail messages: the number of voice mail messages the user has sent
- total day minutes: total number of minutes the user has been in calls during the day
- total day calls: total number of calls the user has done during the day
- total day charge: total amount of money the user was charged by the Telecom company for calls during the day
- total eve minutes: total number of minutes the user has been in calls during the evening
- total eve calls: total number of calls the user has done during the evening
- total eve charge: total amount of money the user was charged by the Telecom company for calls during the evening
- total night minutes: total number of minutes the user has been in calls during the night
- total night calls: total number of calls the user has done during the night
- total night charge: total amount of money the user was charged by the Telecom company for calls during the night
- total intl minutes: total number of minutes the user has been in international calls
- total intl calls: total number of international calls the user has done
- total intl charge: total amount of money the user was charged by the Telecom company for international calls
- customer service calls: number of customer service calls the user has done
- churn: true if the user terminated the contract, otherwise false

Customer churn is the loss of clients or customers. Predicting churn can help the Telecom company, so it can effectively focus a customer retention marketing program (e.g. a special offer) to the subset of clients which are most likely to change their carrier. Therefore, the “churn” column is chosen as target and the following predictive analysis is a supervised classification problem.

The Analysis was conducted in Python 3.7.0 using Jupyter Notebook which is a web application that allows you to create an interactive environment that contains live code, visualizations and text.
In addition, the following packages were used:
- pandas: pandas provides high-performance data structures and operations for manipulating numerical tables and time series.
- numpy: NumPy provides scientific computing capabilities such as a powerful N-dimensional array object, linear algebra, and random number capabilities.
- sklearn: scikit-learn provides tools for data mining and data analysis.
- imblearn: imbalanced-learn provides a number of re-sampling techniques commonly used in datasets showing strong between-class imbalance.
- plotly: Plot.ly is a graphing library which can produce interactive graphs.

In [1]:
# scientific computing libaries
import pandas as pd
import numpy as np

# data mining libaries
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.decomposition import PCA#, FastICA
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, learning_curve
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, roc_auc_score

from imblearn.pipeline import make_pipeline, Pipeline
from imblearn.over_sampling import SMOTE

#plot libaries
import plotly
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True) # to show plots in notebook

# online plotly
from plotly.plotly import plot, iplot
plotly.tools.set_credentials_file(username='XXXXXXXXXXXXXX', api_key='XXXXXXXXXXXXXX')

# offline plotly
#from plotly.offline import plot, iplot

# do not show any warnings
import warnings
warnings.filterwarnings('ignore')

SEED = 17 # specify seed for reproducable results
pd.set_option('display.max_columns', None) # prevents abbreviation (with '...') of columns in prints

In [2]:
RANDOM_FOREST_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # just sqrt is used because values of log2 and sqrt are very similar for our number of features (10-19) 
    'clf__criterion': ['gini', 'entropy'],
    'clf__n_estimators': [100, 300, 500, 1000]
}

DECISION_TREE_PARAMS = {
    'clf__max_depth': [25, 50, 75],
    'clf__max_features': ["sqrt"], # just sqrt is used because values of log2 and sqrt are very similar for our number of features (10-19)
    'clf__criterion': ['gini', 'entropy'],
    'clf__min_samples_split': [6, 10, 14],
}

LOGISTIC_REGRESSION_PARAMS = {
    'clf__solver': ['liblinear'],
    'clf__C': [0.1, 1, 10],
    'clf__penalty': ['l2', 'l1']
}

KNN_PARAMS = {
    'clf__n_neighbors': [5, 15, 25, 35, 45, 55, 65],
    'clf__weights': ['uniform', 'distance'],
    'clf__p': [1, 2, 10]
}

KNN_PARAMS_UNIFORM = {
    'clf__n_neighbors': [5, 15, 25, 35, 45, 55, 65],
    'clf__weights': ['uniform'],
    'clf__p': [1, 2, 10]
}

SVM_PARAMS = [
{
    'clf__kernel': ['linear'],
    'clf__C': [0.1, 1, 10],
}, 
{
    'clf__kernel': ['rbf'],
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__gamma': [0.01, 0.1, 1, 10, 100],
}]

## Exploration ##

In [3]:
# load the dataset
df = pd.read_csv("/Users/Alex/Downloads/churn_kaggle.csv")

print("The dataset has %d rows and %d columns." % df.shape)

The dataset has 3333 rows and 21 columns.


In [4]:
# check for null values in the dataset
print("There are " + ("some" if df.isnull().values.any() else "no")  + " null/missing values in the dataset.")

There are no null/missing values in the dataset.


First, we take a look at some data points to get a feeling what the values of the various columns look like.

In [5]:
df.head(3)

,state,account length,area code,phone number,international plan,voice mail plan,number vmail messages,total day minutes,total day calls,total day charge,total eve minutes,total eve calls,total eve charge,total night minutes,total night calls,total night charge,total intl minutes,total intl calls,total intl charge,customer service calls,churn
0,KS,128,415,382-4657,no,yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,371-7191,no,yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,358-1921,no,no,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False


We can see that the columns "state", "international plan", "voice mail plan" and "churn" have String values. The latter three seem to have just the values "yes" or "no" and are therefore converted to 1 and 0 respectively.

The "state" column is converted using the LabelEncoder, which replaces each unique label with a unique integer.
In this case, a label encode is used instead of dummy variables because of the many distinct values, which when converted into dummy variables would mess up the for example the PCA and the feature importance of the tree-based models.

The "phone number" column is removed, because every customer has its own phone number.

In [6]:
def preprocess_data(df):
    pre_df = df.copy()
    
    # Replace the spaces in the column names with underscores
    pre_df.columns = [s.replace(" ", "_") for s in pre_df.columns]
    
    # convert string columns to integers
    pre_df["international_plan"] = pre_df["international_plan"].apply(lambda x: 0 if x=="no" else 1)
    pre_df["voice_mail_plan"] = pre_df["voice_mail_plan"].apply(lambda x: 0 if x=="no" else 1)
    pre_df = pre_df.drop(["phone_number"], axis=1)
    le = LabelEncoder()
    le.fit(pre_df['state'])
    pre_df['state'] = le.transform(pre_df['state'])
    
    return pre_df, le

In [7]:
pre_df, _ = preprocess_data(df)
pre_df.head(3)

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls,churn
0,16,128,415,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,35,107,415,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,31,137,415,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False


**Statistical overview of the data**

The following statistical measures can be seen for each column using the describe-function of DataFrame of the pandas library:
- count: number of samples
- mean: the mean of this attribute among all samples
- std: the standard deviation of this attribute
- min: the minimal value of this attribute
- 25%: the lower percentile
- 50%: the median
- 75%: the upper percentile
- max: the maximal value of this attribute

In [8]:
pre_df.describe()

,state,account_length,area_code,international_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,customer_service_calls
count,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000,3333.000000
mean,26.059406,101.064806,437.182418,0.096910,0.276628,8.099010,179.775098,100.435644,30.562307,200.980348,100.114311,17.083540,200.872037,100.107711,9.039325,10.237294,4.479448,2.764581,1.562856
std,14.824911,39.822106,42.371290,0.295879,0.447398,13.688365,54.467389,20.069084,9.259435,50.713844,19.922625,4.310668,50.573847,19.568609,2.275873,2.791840,2.461214,0.753773,1.315491
min,0.000000,1.000000,408.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,23.200000,33.000000,1.040000,0.000000,0.000000,0.000000,0.000000
25%,14.000000,74.000000,408.000000,0.000000,0.000000,0.000000,143.700000,87.000000,24.430000,166.600000,87.000000,14.160000,167.000000,87.000000,7.520000,8.500000,3.000000,2.300000,1.000000
50%,26.000000,101.000000,415.000000,0.000000,0.000000,0.000000,179.400000,101.000000,30.500000,201.400000,100.000000,17.120000,201.200000,100.000000,9.050000,10.300000,4.000000,2.780000,1.000000
75%,39.000000,127.000000,510.000000,0.000000,1.000000,20.000000,216.400000,114.000000,36.790000,235.300000,114.000000,20.000000,235.300000,113.000000,10.590000,12.100000,6.000000,3.270000,2.000000
max,50.000000,243.000000,510.000000,1.000000,1.000000,51.000000,350.800000,165.000000,59.640000,363.700000,170.000000,30.910000,395.000000,175.000000,17.770000,20.000000,20.000000,5.400000,9.000000


These numbers are hard to interpret in this format, so we create some graphs which visualize them in a better way. First, we look at the distribution of the our target variable:

In [9]:
colors = plotly.colors.DEFAULT_PLOTLY_COLORS
churn_dict = {0: "no churn", 1: "churn"}

In [10]:
y = df["churn"].value_counts()

data = [go.Bar(x=[churn_dict[x] for x in y.index], y=y.values, marker = dict(color = colors[:len(y.index)]))]
layout = go.Layout(
    title='Churn distribution',
    autosize=False,
    width=400,
    height=400,
    yaxis=dict(
        title='#samples',
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar3')

In [11]:
churn_perc = df["churn"].sum() * 100 / df["churn"].shape[0]
print("Churn percentage is %.3f%%." % churn_perc)

Churn percentage is 14.491%.


We can see that we have clearly more samples for customers without churn than for customers with churn. So we have a class imbalance for the target variable which could lead to predictive models which are biased towards the majority (i.e. no churn). In order to deal with this issue we will investigate into the use of oversampling when building the models.

Next, we look at the churn distribution per state, to see how much the state influences our target:

In [12]:
state_churn_df = df.groupby(["state", "churn"]).size().unstack()
trace1 = go.Bar(
    x=state_churn_df.index,
    y=state_churn_df[0],
    marker = dict(color = colors[0]),
    name='no churn'
)
trace2 = go.Bar(
    x=state_churn_df.index,
    y=state_churn_df[1],
    marker = dict(color = colors[1]),
    name='churn'
)
data = [trace1, trace2]
layout = go.Layout(
    title='Churn distribution per state',
    autosize=True,
    barmode='stack',
    margin=go.layout.Margin(l=50, r=50),
    xaxis=dict(
        title='state',
        tickangle=45
    ),
    yaxis=dict(
        title='#samples',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

We can see that some states have less proportion of customer with churn like AK, HI, IA and some have a higher proportion such as WA, MD and TX. This shows that we should incorporate the state into our further analysis, because it could be help to predict if a customer is going to churn.

The following interactive graph shows the distribution of each feature for customer with churn and for the ones without churn. The slider can be used to switch between the different features.

In [13]:
churn = pre_df[pre_df["churn"] == 1]
no_churn = pre_df[pre_df["churn"] == 0]

In [14]:
def create_churn_trace(col, visible=False):
    return go.Histogram(
        x=churn[col],
        name='churn',
        marker = dict(color = colors[1]),
        visible=visible,
    )

def create_no_churn_trace(col, visible=False):
    return go.Histogram(
        x=no_churn[col],
        name='no churn',
        marker = dict(color = colors[0]),
        visible = visible,
    )

features_not_for_hist = ["state", "phone_number", "churn"]
features_for_hist = [x for x in pre_df.columns if x not in features_not_for_hist]
active_idx = 0
traces_churn = [(create_churn_trace(col) if i != active_idx else create_churn_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
traces_no_churn = [(create_no_churn_trace(col) if i != active_idx else create_no_churn_trace(col, visible=True)) for i, col in enumerate(features_for_hist)]
data = traces_churn + traces_no_churn

n_features = len(features_for_hist)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = features_for_hist[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='#samples',
        automargin=True,
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='histogram_slider')

One interesting histogram is of the feature "international_plan". While the proportion of churn for customers which have the international plan is much higher than the proportion of churn for customers without.

The histograms for the "total_day_minutes" and "total_day_charge" are very similar and we can see that the customer with a higher value for these two features are more likely to churn. Interestingly, this does not apply to the number of day calls, which means that these customers seem to do longer calls. The minutes, charge and #calls for other times of the day (i.e. evening, night) do not show different distributions for customers with churn and without churn.

Another interesting pattern is shown by the "total_intl_calls" feature. The data for the customers with churn are more left skewed than the data of the customers of the customer who did not churn.

Next, we take a look at the box plots for each feature. A box plot visualizes the following statistics:
- median
- the first quartile (Q1) and the third quartile (Q3) building the interquartile range (IQR)
- the lower fence (Q1 - 1.5 * IQR) and the upper fence (Q3 + 1.5 * IQR)
- the maximum and the minimum value

In [15]:
def create_box_churn_trace(col, visible=False):
    return go.Box(
        y=churn[col],
        name='churn',
        marker = dict(color = colors[1]),
        visible=visible,
    )

def create_box_no_churn_trace(col, visible=False):
    return go.Box(
        y=no_churn[col],
        name='no churn',
        marker = dict(color = colors[0]),
        visible = visible,
    )

features_not_for_hist = ["state", "phone_number", "churn"]
features_for_hist = [x for x in pre_df.columns if x not in features_not_for_hist]
# remove features with too less distinct values (e.g. binary features), because boxplot does not make any sense for them
features_for_box = [col for col in features_for_hist if len(churn[col].unique())>5]

active_idx = 0
box_traces_churn = [(create_box_churn_trace(col) if i != active_idx else create_box_churn_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
box_traces_no_churn = [(create_box_no_churn_trace(col) if i != active_idx else create_box_no_churn_trace(col, visible=True)) for i, col in enumerate(features_for_box)]
data = box_traces_churn + box_traces_no_churn

n_features = len(features_for_box)
steps = []
for i in range(n_features):
    step = dict(
        method = 'restyle',  
        args = ['visible', [False] * len(data)],
        label = features_for_box[i],
    )
    step['args'][1][i] = True # Toggle i'th trace to "visible"
    step['args'][1][i + n_features] = True # Toggle i'th trace to "visible"
    steps.append(step)

sliders = [dict(
    active = active_idx,
    currentvalue = dict(
        prefix = "Feature: ", 
        xanchor= 'center',
    ),
    pad = {"t": 50},
    steps = steps,
    len=1,
)]

layout = dict(
    sliders=sliders,
    yaxis=dict(
        title='value',
        automargin=True,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)

fig = dict(data=data, layout=layout)

iplot(fig, filename='box_slider')

When we look at the box plot for the number of voice mail messages ("number_vmail_messages"), we can see that we have some outliers for the customers with churn, but most of them have send zero voice mail messages. The customers which did not churn instead tend to do more voice mail messages.<br/>
Similar to our findings in the histograms, we can see also in the box plot that the median of the total day minutes and the total day charge for churn clients is higher than the one of no-churn clients.<br/>
Looking at the total international calls ("total_intl_calls"), the box plot shows that both churn and no-churn  customers are doing a similar amount of international calls, but the churn-customers tend to do longer calls as the median of churn customers for the total international minutes is higher than for the no-churn customers.<br/>
Finally, the plot for the number of customer service calls shows that clients with churn have a higher median and a higher variance for the customer service calls.

In order to investigate the pair-wise correlations between two variables $X$ and $Y$, we use the Pearson correlation. Let $\sigma_X, \sigma_Y$ be the standard deviation of X,Y and $\text{cov}(X, Y) = E[(X-E[X])(Y-E[Y])]$. Then we can define the Pearson correlation as the following:<br/>
$\rho_{X, Y} = \frac{\text{cov}(X, Y)}{\sigma_X \sigma_Y}\,$.

To visualize these correlations we use a heatmap plot, in which high correlations are coloured more to the red and lower ones more to the blue.

In [16]:
corr = pre_df.corr()
trace = go.Heatmap(z=corr.values.tolist(), x=corr.columns, y=corr.columns)
data=[trace]
layout = go.Layout(
    title='Heatmap of pairwise correlation of the columns',
    autosize=False,
    width=850,
    height=700,
    yaxis=go.layout.YAxis(automargin=True),
    xaxis=dict(tickangle=40),
    margin=go.layout.Margin(l=0, r=200, b=200, t=80)
)


fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='labelled-heatmap1')

We can see a high correlation between the voice mail plan and the number of voice mail messages. It makes sense that customers with the voice mail plan also send more voice mail messages.<br/>
However, the international plan is just slightly correlated with the total international minutes and the international charge.<br/>
As seen also in our previous analysis, the total day charge and the total day minutes a very highly correlated. Probably, this Telecom company charges per minute. The same behavior can be seen for the evening, the night and the international calls.<br/>
The highest correlation with the churn variable have the international plan, the total_day_charge, the total_day_minutes and the number of customer service calls.

In order to reduce the dimensionality of our dataset, we can identify and remove duplicate features according to their pairwise correlation with others. For this, we conduct a clustering of the features using agglomerative hierarchical clustering with average linkage. This method starts by creating one cluster for each feature and by computing the pairwise distance/dissimilarity/similarity between all the clusters, which in our case is the correlation. Then it select the two clusters with the highest average correlation to be merged. In the next iteration, the next pair of clusters is selected to be merged. This process is repeated until we end up with one cluster.

To visualize the clustering process a dendrogram is shown in the following:

In [78]:
from scipy.cluster import hierarchy as hc
X = np.random.rand(10, 10)
names = pre_df.columns
inverse_correlation = 1 - abs(pre_df.corr())
fig = ff.create_dendrogram(inverse_correlation.values, orientation='left', labels=names, colorscale=colors, linkagefun=lambda x: hc.linkage(x, 'average'))
fig['layout'].update(dict(
    title="Dendrogram of clustering the features according to correlation",
    width=800, 
    height=600,
    margin=go.layout.Margin(l=180, r=50),
    xaxis=dict(
        title='distance',
    ),
    yaxis=dict(
        title='features',
        automargin=True,
    ),
))
iplot(fig, filename='dendrogram_corr_clustering')

The four feature pairs which are each clustered together at a very low distance are:
- total_night_minutes and total_night_charge
- total_eve_minutes and total_eve_charge
- total_intl_minutes and total_intl_charge
- total_day_minutes and total_day_charge

We save the all the charge features as duplicate features to be removed for our reduced dataset.

In [18]:
# save the duplicate features for later usage
duplicate_features = ["total_day_charge", "total_eve_charge", "total_night_charge", "total_intl_charge"]

To prepare the dataset for the following analysis we split it into the target column and the other predictors. In addition, we standardize all features to avoid e.g. higher impact of features with higher absolute values in classifiers which are based on a distance metric.

In [19]:
# splitting the dataset into feature vectors and the target variable
df_y = pre_df["churn"]
df_X = pre_df.drop(["churn"], axis=1)

In [20]:
# normalize the dataset (note: for decision tree/random forest it would not be needed)
df_X_normed = (df_X - df_X.mean()) / df_X.std()

## Principal component analysis ##

Principal component analysis (PCA) is a statistical procedure that uses an orthogonal transformation to convert the variables of a dataset into a new set of variables which are linearly uncorrelated and called principal components. The principal components are ranked according to the variance of data along them. This technique can be used to reduce the dimensionality of the dataset by considering just the most important principal components.

In order to reduce the dimensionality of a dataset $X$ with $n$ variables to a new dataset with $k$ variables using PCA the following steps have to be followed:
* standardize the data
* calculate the Eigenvectors and Eigenvalues from the covariance matrix or correlation matrix, or perform Singular Vector Decomposition.
* sort the Eigenvectors according to their Eigenvalues in decreasing order
* build the $n×k$-dimensional projection matrix $W$ by putting the top $k$ Eigenvectors into the columns of $W$
* transform the dataset $X$ by multiplying it with $W$: $X_{t} = XW$

We apply PCA to the dataset in order to reduce the number of features.

In [21]:
# calculate the principal components
pca = PCA(random_state=SEED)
df_X_pca = pca.fit_transform(df_X_normed)

In [22]:
tot = sum(pca.explained_variance_) # total explained variance of all principal components
var_exp = [(i / tot) * 100 for i in sorted(pca.explained_variance_, reverse=True)] # individual explained variance
cum_var_exp = np.cumsum(var_exp) # cumulative explained variance

In the following we plot the Scree Plot to determine how many components we use.

In [23]:
trace_cum_var_exp = go.Bar(
    x=list(range(1, len(cum_var_exp) + 1)), 
    y=var_exp,
    name="individual explained variance",
)
trace_ind_var_exp = go.Scatter(
    x=list(range(1, len(cum_var_exp) + 1)),
    y=cum_var_exp,
    mode='lines+markers',
    name="cumulative explained variance",
    line=dict(
        shape='hv',
    ))
data = [trace_cum_var_exp, trace_ind_var_exp]
layout = go.Layout(
    title='Individual and Cumulative Explained Variance',
    autosize=True,
    yaxis=dict(
        title='percentage of explained variance',
    ),
    xaxis=dict(
        title="principal components",
        dtick=1,
    ),
    legend=dict(
        x=0,
        y=1,
    ),
)
fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='basic-bar')

We can see from the graph that the first five components explain the most individual variance, followed by the next nine components which explain less variance. The last five components explain near no variance. We choose to reduce our dataset by using the first ten components which explain about 80% of the total variance.

In [24]:
n_components = 10
df_X_reduced = np.dot(df_X_normed.values, pca.components_[:n_components,:].T)
df_X_reduced = pd.DataFrame(df_X_reduced, columns=["PC#%d" % (x + 1) for x in range(n_components)])

## Comparison of differently preprocessed datasets for classification ##

In this chapter we apply one classifier to the different version of the dataset. The Random Forest classifier is used because it is considered as a great baseline model for most applications. It is described in more detail in the next chapter. The following different versions of the dataset are investigated:
- full dataset
- dataset with variables reduced by the clustering according to the correlation
- dataset reduced by considering the first ten principal components after applying PCA

Furthermore, we apply two different methods to deal with the unbalanced target variable. First, we try adjusting the weights for the penalization when the classifier makes a mistake in the training and then we try to oversample the data using the Synthetic Minority Over-sampling Technique.

For the evaluation of the classifiers on the different datasets, a hold-out test set is used, which has 20% of all the data. To account for the class imbalance of our target variable, we use the f1-score as our main evaluation metric.
We define:
- TP = #samples for which the prediction is positive and the true label is positive
- FP = #samples for which the prediction is positive but the true label is negative
- TN = #samples for which the prediction is negative and the true label is negative
- FN = #samples for which the prediction is negative but the true label is positive

Then we define the following:

$\text{precision} = \frac{TP}{TP + FP} \;\;\; \text{and} \;\;\; \text{recall} = \frac{TP}{TP + FN}$

Then the f1-score is given by the following equation:

$F_{1}=2\,\frac{\text{precision}\, \times \,\text{recall}}{\text{precision} \, +\, \text{recall}}$

Every classifier has a set of hyperparameters, which can be tuned by training the classifier with different values for these hyperparameters and selecting the classifier with the best score. In order to estimate the performance of a classifier in a more reliable way, k-fold cross validation (CV) is used. In k-fold CV, the training set is divided into a k subsets. Then we train k times our classifier on different unions of k-1 subsets and calculate its score on the subset which was not used for training. Then the final score is calculated by averaging the score of each iteration. In detail, let $C_1, C_2, ... C_k$ be the indices of the samples in each of the $K$ parts of the dataset and let $n_k$ be the number of observations in part $k$. Then the score from the cross validation is computed as follows:

$\text{Score}_{CV(K)} = \sum_{k=1}^{K} \frac{n_k}{n}\text{Score}_k$.

In our hyper parameter tuning the Score is the f1-Score defined above.

To do this analysis, we use the sklearn.model_selection.GridSearchCV object, to which we pass a classifier, a dictionary of hyperparameters with values and a $k$-fold object. For a good trade-off between runtime and accuracy of the score we choose $k=5$, so the classifiers are trained on 80% of the train data in each iteration.

In [25]:
# prints the best grid search scores along with their parameters.
def print_best_grid_search_scores_with_params(grid_search, n=5):
    if not hasattr(grid_search, 'best_score_'):
        raise KeyError('grid_search is not fitted.')
    print("Best grid scores on validation set:")
    indexes = np.argsort(grid_search.cv_results_['mean_test_score'])[::-1][:n]
    means = grid_search.cv_results_['mean_test_score'][indexes]
    stds = grid_search.cv_results_['std_test_score'][indexes]
    params = np.array(grid_search.cv_results_['params'])[indexes]
    for mean, std, params in zip(means, stds, params):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

In [26]:
def do_gridsearch_with_cv(clf, params, X_train, y_train, cv, smote=None):

    if smote is None:
        pipeline = Pipeline([('clf', clf)])
    else:
        pipeline = Pipeline([('sm', sm), ('clf', clf)])
        
    gs = GridSearchCV(pipeline, params, cv=kf, n_jobs=-1, scoring='f1', return_train_score=True)
    gs.fit(X_train, y_train)
    return gs

def score_on_test_set(clfs, datasets):
    scores = []
    for c, (X_test, y_test) in zip(clfs, datasets):
        scores.append(c.score(X_test, y_test))
    return scores

In [27]:
# split data into train and test set in proportion 4:1 for all differntly preprocessed datasets
X_train, X_test, y_train, y_test = train_test_split(df_X_normed, df_y, test_size=0.2, random_state=SEED)
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(df_X_reduced, df_y, test_size=0.2, random_state=SEED)
cols_without_duplicate = [x for x in df_X_normed.columns if x not in duplicate_features]
X_train_red, X_test_red, y_train_red, y_test_red = train_test_split(df_X_normed[cols_without_duplicate], df_y, test_size=0.2, random_state=SEED)

In [28]:
print("Shape of the full train dataset:", X_train.shape)
print("Shape of the train dataset with reduced features", X_train_red.shape)
print("Shape of the transformed train dataset using the first 10 Principal Components", X_train_pca.shape)

Shape of the full train dataset: (2666, 19)
Shape of the train dataset with reduced features (2666, 15)
Shape of the transformed train dataset using the first 10 Principal Components (2666, 10)


In [29]:
sm = SMOTE(random_state=SEED)
kf = StratifiedKFold(n_splits=5, random_state=SEED)
clf_rf = RandomForestClassifier(random_state=SEED)
clf_balanced = RandomForestClassifier(random_state=SEED, class_weight="balanced")

### Without additional balancing techniques ###

We train the Random Forest on the full dataset, on the dataset with reduced features and on the dataset transformed using the first ten principal components.

In [30]:
%%time
gs_full = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=None)
gs_red = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=None)
gs_pca = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=None)
gss_raw = [gs_full, gs_red, gs_pca]

CPU times: user 14.9 s, sys: 342 ms, total: 15.2 s
Wall time: 6min 22s


In [31]:
test_results_raw = score_on_test_set(gss_raw, [(X_test, y_test), (X_test_red, y_test_red), (X_test_pca, y_test_pca)])

### Using class weights in the loss function ###

The sklearn library offers for all parametric classifiers a parameter class_weight, which can be set to "balanced". Then, mistakes are weighted inversely proportional to the class frequencies. This means that mistakes for the minority class are penalized more than mistakes made for the majority class.

In [32]:
%%time
gs_full_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=None)
gs_red_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=None)
gs_pca_balanced = do_gridsearch_with_cv(clf_balanced, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=None)
gss_balanced_weights = [gs_full_balanced, gs_red_balanced, gs_pca_balanced]

CPU times: user 14.4 s, sys: 198 ms, total: 14.6 s
Wall time: 5min 46s


In [33]:
test_results_balanced_weights = score_on_test_set(gss_balanced_weights, [(X_test, y_test), (X_test_red, y_test_red), (X_test_pca, y_test_pca)])

### Using Synthetic Minority Over-sampling Technique ###

The Synthetic Minority Over-sampling Technique (SMOTE) algorithm applies KNN approach where it selects one of the k nearest neighbors and computes the vector between the original point and the selected neighbor. The difference is multiplied by random number between (0, 1) and it is added back to original point to obtain the new synthetic point. Geometrically, the synthetic point is somewhere on the line between the original point and its neighbor.

In the following, we use the SMOTE implementation SMOTE of the imblearn.over_sampling library. Furthermore, we create a Pipeline of applying Smote and then training the classifier, so that it is executed in every fold of x-fold cross validation.

In [34]:
%%time
gs_full_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=sm)
gs_red_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_red, y_train_red, kf, smote=sm)
gs_pca_smote = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train_pca, y_train_pca, kf, smote=sm)
gss_smote = [gs_full_smote, gs_red_smote, gs_pca_smote]

CPU times: user 13.9 s, sys: 160 ms, total: 14 s
Wall time: 10min 14s


In [35]:
test_results_smote = score_on_test_set(gss_smote, [(X_test, y_test), (X_test_red, y_test_red), (X_test_pca, y_test_pca)])

### Comparison ###

In [36]:
dataset_strings = ["full dataset", "data set with reduced features", "dataset with first 10 principal components"]
method_strings = ["without any balancing", "using balanced class weights", "using SMOTE"]

result_strings = dict()
for ms, results in zip(method_strings, [test_results_raw, test_results_balanced_weights, test_results_smote]):
    for ds, res in zip(dataset_strings, results):
        string = "%.3f" % res + "     " + ds + " " + ms
        result_strings[string] = res
        2
result_strings = sorted(result_strings.items(), key=lambda kv: kv[1], reverse=True)
print("F1 score  dataset and method")
for k, _ in result_strings:
    print(k)

F1 score  dataset and method
0.821     full dataset using SMOTE
0.810     data set with reduced features using SMOTE
0.805     full dataset without any balancing
0.787     full dataset using balanced class weights
0.767     data set with reduced features without any balancing
0.729     data set with reduced features using balanced class weights
0.602     dataset with first 10 principal components using SMOTE
0.500     dataset with first 10 principal components without any balancing
0.496     dataset with first 10 principal components using balanced class weights


The classifier trained on the PCA-transformed dataset performs the worst. The best results are obtained using the full dataset with SMOTE, closely followed by the reduced features dataset with SMOTE. So using SMOTE achieved better results than applying balanced class weights.

In conclusion, we use the full dataset and apply SMOTE in the following classification chapter.

## Classification ##

In [37]:
def get_color_with_opacity(color, opacity):
    return "rgba(" + color[4:-1] + ", %.2f)" % opacity

# partially based on https://scikit-learn.org/stable/auto_examples/model_selection/plot_learning_curve.html
def plot_learning_curve(estimator, title, X, y, cv=None, n_jobs=-1, train_sizes=np.linspace(.1, 1.0, 5)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring="f1", random_state=SEED)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    trace1 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean - train_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
    )
    trace2 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean + train_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[0], 0.4),
        ),
    )
    trace3 = go.Scatter(
        x=train_sizes, 
        y=train_scores_mean, 
        showlegend=True,
        name="Train score",
        line = dict(
            color = colors[0],
        ),
    )
    
    trace4 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean - test_scores_std, 
        showlegend=False,
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
    )
    trace5 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean + test_scores_std, 
        showlegend=False,
        fill="tonexty",
        mode="lines",
        name="",
        hoverlabel = dict(
            namelength=20
        ),
        line = dict(
            width = 0.1,
            color = get_color_with_opacity(colors[1], 0.4),
        ),
    )
    trace6 = go.Scatter(
        x=train_sizes, 
        y=test_scores_mean, 
        showlegend=True,
        name="Test score",
        line = dict(
            color = colors[1],
        ),
    )
    
    data = [trace1, trace2, trace3, trace4, trace5, trace6]
    layout = go.Layout(
        title=title,
        autosize=True,
        yaxis=dict(
            title='F1 Score',
        ),
        xaxis=dict(
            title="#Training samples",
        ),
        legend=dict(
            x=0.8,
            y=0,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

In [38]:
def plot_feature_importance(feature_importance, title):
    trace1 = go.Bar(
        x=feature_importance[:, 0],
        y=feature_importance[:, 1],
        marker = dict(color = colors[0]),
        name='feature importance'
    )
    data = [trace1]
    layout = go.Layout(
        title=title,
        autosize=True,
        margin=go.layout.Margin(l=50, r=100, b=150),
        xaxis=dict(
            title='feature',
            tickangle=30
        ),
        yaxis=dict(
            title='feature importance',
            automargin=True,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return iplot(fig, filename=title)

### Logistic Regression ###

We define $x_i$ as the $n$-dimensional feature vector of a given sample and $\beta_{0},\,\boldsymbol{\beta} = (\beta_{1}, ..., \beta_{n})^T$ as the model parameters. Then the logistic regression model is defined as:

$P(Y=1 \vert x_i)= \frac{\text{exp}(\beta_{0} + x_i^T\boldsymbol{\beta} )}{1+\text{exp}(\beta_{0} + x_i^T\boldsymbol{\beta} )} = \frac{1}{1+\text{exp}(-(\beta_{0} + x_i^T\boldsymbol{\beta} ))}$

The hyperparameters of a logistic regression include the following ones, which can be passed to the LogisticRegression of sklearn.linear_model:
- penalty: the norm used for penalization (default='l2')
- C: the inverse of the regularization strength (default=1.0)

In [39]:
%%time
clf_lr = LogisticRegression(random_state=SEED)
gs_lr = do_gridsearch_with_cv(clf_lr, LOGISTIC_REGRESSION_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 275 ms, sys: 12.1 ms, total: 287 ms
Wall time: 2.34 s


In [40]:
print_best_grid_search_scores_with_params(gs_lr)

Best grid scores on validation set:
0.493 (+/-0.051) for {'clf__C': 10, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.493 (+/-0.050) for {'clf__C': 10, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.492 (+/-0.048) for {'clf__C': 1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}
0.492 (+/-0.047) for {'clf__C': 1, 'clf__penalty': 'l1', 'clf__solver': 'liblinear'}
0.491 (+/-0.044) for {'clf__C': 0.1, 'clf__penalty': 'l2', 'clf__solver': 'liblinear'}


In [41]:
gs_lr_score = gs_lr.score(X_test, y_test)
y_pred_lr = gs_lr.predict(X_test)
cm_lr = confusion_matrix(y_test, y_pred_lr)
cm_lr = cm_lr.astype('float') / cm_lr.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In the following, the normalized confusion matrix is shown. It demonstrates the proportion of samples which are true churn and predicted as churn/no churn and the proportion of samples which are true no churn and predicted as churn/no churn.

In [42]:
cm_df = pd.DataFrame(cm_lr.round(3), index=["true no churn", "true churn"], columns=["predicted no churn", "predicted churn"])
cm_df

,predicted no churn,predicted churn
true no churn,0.756,0.244
true churn,0.212,0.788


The confusion matrix of the Logistic Regression shows that the probability to predict the correct class is with 0.76 and 0.79 similar for both classes. This means that the Logistic Regression has just a slight bias towards predicting a customer as churn.

In [43]:
plot_learning_curve(gs_lr.best_estimator_, "Learning Curve of Logistic Regression", X_train, y_train, cv=5)

For the learning curve of the Logistic Regression on the train set we can not see a clear trend, but on the train set the score increases with the number of training examples. Trained on all training samples the f1-score on the train set and on the test set is very similar, so the Logistic Regression is not overfitting the train data.

### K-Nearest Neighbors ###

The K-Nearest Neighbors algorithm (KNN) is a non-parametric method, which considers the K closest training examples to the point of interest for predicting its class. This is done by a simple majority vote over the K closest points.

The hyperparameters of KNN include the following ones, which can be passed to the KNeighborsClassifier of sklearn.neighbors:
- n_neighbors: corresponds to K, the number of nearest neighbors considered for the prediction (default=5) 
- weights: 
  - if uniform, then all neighbors have the same weight for the voting (default)
  - if distance, then the votes of the neighbors are weighted by the inverse of the distance for the voting
- p: the power parameter for the Minkowski metric (default=2)

In [44]:
%%time
clf_knn = KNeighborsClassifier()
gs_knn = do_gridsearch_with_cv(clf_knn, KNN_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 1.07 s, sys: 61.1 ms, total: 1.13 s
Wall time: 2min 4s


In [45]:
print_best_grid_search_scores_with_params(gs_knn)

Best grid scores on validation set:
0.604 (+/-0.079) for {'clf__n_neighbors': 55, 'clf__p': 1, 'clf__weights': 'distance'}
0.603 (+/-0.080) for {'clf__n_neighbors': 45, 'clf__p': 1, 'clf__weights': 'uniform'}
0.602 (+/-0.090) for {'clf__n_neighbors': 55, 'clf__p': 1, 'clf__weights': 'uniform'}
0.600 (+/-0.079) for {'clf__n_neighbors': 45, 'clf__p': 1, 'clf__weights': 'distance'}
0.599 (+/-0.088) for {'clf__n_neighbors': 65, 'clf__p': 1, 'clf__weights': 'distance'}


In [46]:
gs_knn_score = gs_knn.score(X_test, y_test)
y_pred_knn = gs_knn.predict(X_test)
cm_knn = confusion_matrix(y_test, y_pred_knn)
cm_knn = cm_knn.astype('float') / cm_knn.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In the following, the normalized confusion matrix is shown:

In [47]:
cm_df = pd.DataFrame(cm_knn.round(3), index=["true no churn", "true churn"], columns=["predicted no churn", "predicted churn"])
cm_df

,predicted no churn,predicted churn
true no churn,0.864,0.136
true churn,0.176,0.824


The KNN classifier shows a small bias towards predicting no churn.

In [48]:
plot_learning_curve(gs_knn.best_estimator_, "Learning Curve of KNN", X_train, y_train, cv=5)

The f1-score of 1 on the train set is a bit strange on the first sight. This behavior is caused by using the "distance" weights for the vote of the neighbors. This means that in the majority vote every point has a vote weight equal to the inverse of its distance to the point of interest. Because the point of interest is in the train set the distance to the nearest point is 0. So the weight of the vote of this point is infinite and the prediction is therefore equal to the class of the point itself. This applies to all training samples which results in a f1-score of 1.

To show a more interesting learning curve on the train set, we train another KNN classifier using uniform weights instead of the distance weights:

In [49]:
clf_knn_uni = KNeighborsClassifier()
gs_knn_uniform = do_gridsearch_with_cv(clf_knn_uni, KNN_PARAMS_UNIFORM, X_train, y_train, kf, smote=sm)

In [50]:
print_best_grid_search_scores_with_params(gs_knn_uniform, 1)

Best grid scores on validation set:
0.603 (+/-0.080) for {'clf__n_neighbors': 45, 'clf__p': 1, 'clf__weights': 'uniform'}


In [51]:
plot_learning_curve(gs_knn_uniform.best_estimator_, "Learning Curve of KNN with uniform weights", X_train, y_train, cv=5)

The learning curve shows that the f1-score on the test set constantly increases, which makes sense because the KNN can consider more training examples and therefore it can do a more accurate prediction on the test points. The train score has also an increasing trend but does not improve between 692 and 1652 training samples.

### Support-Vector Machine ###

A linear Support-Vector Machine (SVM) finds the optimal hyperplane between the points of two classes such that the distance of the nearest points to the decision boundary is maximized. This distance is called margin.

If the data set is not linearly separable, we can map the samples ${\bf x}$ into a feature space of higher dimensions:
${\bf x} \longrightarrow \phi({\bf x})$ in which the classes can be linearly separated. This results in a non-linear decision boundary in the original dimensions.

As the vectors ${\bf x}_i$ appear only in inner products in both the decision
function and the learning law, the mapping function $\phi({\bf x})$ does not 
need to be explicitly specified. Instead, we define a so-called kernel function:

$ K({\bf x}_1,{\bf x}_2)=\phi({\bf x}_1)^T\phi({\bf x}_2)$.

In the gridsearch we consider the following two kernels:
- linear kernel: $ K({\bf x}_1,{\bf x}_2) = {\bf x}_1 \cdot {\bf x}_2$
- radial basis function: $ K({\bf x}_1,{\bf x}_2) = exp(-\gamma({\Vert {\bf x}_1 - {\bf x}_2 \Vert}^2))$

The hyperparameters of a SVM include the following ones, which can be passed to the SVC of sklearn.svm:
- C: the inverse of the regularization strength (default=1.0)
- kernel: the kernel used (default='rbf')
- gamma: The higher the gamma value it tries to exactly fit the training data set (default='auto_deprecated')

In [52]:
%%time
clf_svm = svm.SVC(random_state=SEED, probability=True)
gs_svm = do_gridsearch_with_cv(clf_svm, SVM_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 3.17 s, sys: 149 ms, total: 3.32 s
Wall time: 3min 14s


In [53]:
print_best_grid_search_scores_with_params(gs_svm)

Best grid scores on validation set:
0.632 (+/-0.059) for {'clf__C': 10, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
0.631 (+/-0.082) for {'clf__C': 0.1, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}
0.620 (+/-0.074) for {'clf__C': 1, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
0.617 (+/-0.091) for {'clf__C': 100, 'clf__gamma': 0.01, 'clf__kernel': 'rbf'}
0.613 (+/-0.108) for {'clf__C': 1, 'clf__gamma': 0.1, 'clf__kernel': 'rbf'}


In [54]:
gs_svm_score = gs_svm.score(X_test, y_test)
y_pred_svm = gs_svm.predict(X_test)
cm_svm = confusion_matrix(y_test, y_pred_svm)
cm_svm = cm_svm.astype('float') / cm_svm.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In the following, the normalized confusion matrix is shown:

In [55]:
pd.DataFrame(cm_svm.round(3), index=["true no churn", "true churn"], columns=["predicted no churn", "predicted churn"])

,predicted no churn,predicted churn
true no churn,0.918,0.082
true churn,0.212,0.788


The confusion matrix shows that the SVM classifier has a clear bias towards predicting no churn. This is not desirable in our case, because we do not want to miss out on churn-customers.

In [56]:
plot_learning_curve(gs_svm.best_estimator_, "Learning Curve of SVM", X_train, y_train, cv=5)

The train score of the SVM does not improve with the number of training samples. The test score, however, improves for the first 692 samples but stays the same with more samples. This could be due to the fact that the decision boundary of the SVM depends just on the support vectors and therefore changes just in the case that the additional training samples are support vectors. Furthermore, the train score is always by more than 0.1 higher than the test score which means that the SVM is overfitting on the train data.

### Decision Tree ###

A decision tree for classification consists of several splits, which determine for a input sample, the predicted class, which is a leaf node in the tree. The construction of the decision trees is done with a greedy algorithm, because the theoretical minimum of function exists but it is NP-hard to determine it, because number of partitions has a factorial growth.
Specifically, a greedy top-down approach is used which chooses a variable at each step that best splits the set of items. For measuring the "best" different metrics can be used, which generally measure the homogeneity of the target variable within the subsets. For this analysis we consider the following two metrics:

Gini impurity: Let $j$ be the number of classes and $p_i$ the fraction of items of class $i$ in a subset $p$, for $i \in \{1,2,..., j\}$. Then the gini impurity is defined as follows: $\;I_G(p) = 1- \sum_{i=1}^j {p_i}^2$.


Information gain: It measures the reduction in entropy when applying the split. The entropy is defined as $H(t) = - \sum_{i=1}^j p_i\, \text{log}_2\,p_i$. Then we define the information gain to split $n$ samples in parent node $p$ into $k$ partitions, where $n_i$ is the number of samples in partition $i$ as $IG = H(p) - \sum_{i = 1}^k \frac{n_i}{n} H(i)$.

The hyperparameters of a Decision Tree include the following ones, which can be passed to the DecisionTreeClassifier of sklearn.tree:
- criterion: the criterion which decides the feature and the value at the split (default='gini')
- max_depth: the maximum depth of each tree (default=None)
- min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)

In [57]:
%%time
clf_dt = DecisionTreeClassifier(random_state=SEED)
gs_dt = do_gridsearch_with_cv(clf_dt, DECISION_TREE_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 250 ms, sys: 23.5 ms, total: 274 ms
Wall time: 1.44 s


In [58]:
print_best_grid_search_scores_with_params(gs_dt)

Best grid scores on validation set:
0.612 (+/-0.072) for {'clf__criterion': 'gini', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.612 (+/-0.072) for {'clf__criterion': 'gini', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.612 (+/-0.072) for {'clf__criterion': 'gini', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.587 (+/-0.033) for {'clf__criterion': 'entropy', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}
0.586 (+/-0.033) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__min_samples_split': 6}


In [59]:
gs_dt_score = gs_dt.score(X_test, y_test)
y_pred_dt = gs_dt.predict(X_test)
cm_dt = confusion_matrix(y_test, y_pred_dt)
cm_dt = cm_dt.astype('float') / cm_dt.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In the following, the normalized confusion matrix is shown:

In [60]:
cm_df = pd.DataFrame(cm_dt.round(3), index=["true no churn", "true churn"], columns=["predicted no churn", "predicted churn"])
cm_df

,predicted no churn,predicted churn
true no churn,0.912,0.088
true churn,0.271,0.729


Similar to the SVM, also the Decision Tree has some bias towards predicting no churn, resulting in a poor performance in classifying the true churn clients.

The Decision Tree class of sklearn also computes an importance value for each feature. This is done by weighting the decrease of impurity at each split by the probability of reaching this node for each node which split involves the feature of interest. Then these weighted decreases of impurity are summed up for each feature and this gives the feature importance.

In [61]:
feature_importance = np.array(sorted(zip(X_train.columns, gs_dt.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance, "Feature importance in the decision tree")

We can see that the most important features for the Decision Tree are the customer service calls and the total day minutes. These were also found to be correlated with the target variable in the data exploration. Interestingly, the area code is the third most important feature, but the area code has near zero correlation with the churn column.

In [62]:
plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Decision Tree", X_train, y_train, cv=5)

The Decision Tree is highly overfitted as the train score is about 0.3 higher than the test score. Both the train and especially the test score have an increasing trend with the number of samples.

### Random Forest ###

A random forest is an ensemble model that fits a number of decision tree classifiers on various sub-samples of the dataset which are created by the use of bootstrapping. In the inference stage it uses a majority vote over all trees to obtain the prediction. This improves the predictive accuracy and controls over-fitting. 

The hyperparameters of a random forest include the following ones, which can be passed to the RandomForestClassifier of sklearn.ensemble:
- n_estimators: the number of trees 
- criterion: the criterion which decides the feature and the value at the split (default='gini')
- max_depth: the maximum depth of each tree (default=None)
- min_samples_split: the minimum number of samples in a node to be considered for further splitting (default=2)
- max_features: the number of features which are considered for a split (default='sqrt')

In [63]:
%%time
clf_rf = RandomForestClassifier(random_state=SEED)
gs_rf = do_gridsearch_with_cv(clf_rf, RANDOM_FOREST_PARAMS, X_train, y_train, kf, smote=sm)

CPU times: user 6.71 s, sys: 115 ms, total: 6.83 s
Wall time: 3min 31s


In [64]:
print_best_grid_search_scores_with_params(gs_rf)

Best grid scores on validation set:
0.789 (+/-0.037) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
0.789 (+/-0.037) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
0.788 (+/-0.044) for {'clf__criterion': 'entropy', 'clf__max_depth': 25, 'clf__max_features': 'sqrt', 'clf__n_estimators': 500}
0.787 (+/-0.032) for {'clf__criterion': 'entropy', 'clf__max_depth': 75, 'clf__max_features': 'sqrt', 'clf__n_estimators': 100}
0.787 (+/-0.032) for {'clf__criterion': 'entropy', 'clf__max_depth': 50, 'clf__max_features': 'sqrt', 'clf__n_estimators': 100}


In [65]:
gs_rf_score = gs_rf.score(X_test, y_test)
y_pred_rf = gs_rf.predict(X_test)
cm_rf = confusion_matrix(y_test, y_pred_rf)
cm_rf = cm_rf.astype('float') / cm_rf.sum(axis=1)[:, np.newaxis] # normalize the confusion matrix

In the following, the normalized confusion matrix is shown:

In [66]:
cm_df = pd.DataFrame(cm_rf.round(3), index=["true no churn", "true churn"], columns=["predicted no churn", "predicted churn"])
cm_df

,predicted no churn,predicted churn
true no churn,0.976,0.024
true churn,0.188,0.812


The Random Forest classifier shows also some bias towards predicting no churn, but not as much as the Decision Tree. Furthermore the results are very good by misclassifying just very few test samples.

In the Random Forest the feature importances are obtained by computing it for all trees and taking the average.

In [67]:
feature_importance_rf = np.array(sorted(zip(X_train.columns, gs_rf.best_estimator_.named_steps['clf'].feature_importances_), key=lambda x: x[1], reverse=True))
plot_feature_importance(feature_importance_rf, "Feature importance in the Random Forest")

The most important feature for the Random Forest are the number of customer service calls, the total day minutes and the total day charge. The latter two basically contain the same information as they are highly correlated. Other important features are if the user has an international plan and the total international calls. The area code was not found as important as in the Decision Tree.

In [68]:
plot_learning_curve(gs_dt.best_estimator_, "Learning Curve of the Random Forest", X_train, y_train, cv=5)

It can be seen that the test score of the Random Forest constantly increases with the number of samples. The train score is with over 0.9 always very high and shows that also the Random Forest overfits on the data.

### Comparison ###

To compare the performance of the different classification model, we consider several evalutation metric in addition to the in chapter 4 defined precision, recall and f1-score. Further, let TP, FP, TN, FN be defined as in chapter 4.

**Accuracy**<br/>
The accuracy is the percentage of samples classified correctly: $\text{accuracy} = \frac{TP + TN}{TP + FP + TN + FN}$.

**Area Under the Receiver Operating Characteristic curve (AUC)**<br/>
To introduce this concept, we define the following two metrics:
- True positive rate (TPR): this is the same as the recall: $FPR = \text{recall} = \frac{TP}{FN + TP}$
- False positive rate (FPR): this corresponds to the proportion of negative data points that are mistakenly considered as positive, with respect to all negative data points: $FPR = \frac{FP}{TN + FP}$

To plot the Receiver Operating Characteristic (ROC) curve we choose a number of different classification thresholds and compute the TPR and the FPR. So the curve shows the trade-off between these two. To combine the TPR and the FPR into one evaluation metric the area under the ROC curve (AUC) is computed.

The following plot shows the ROC curves of the classifiers trained in the previous chapters:

In [69]:
# code partially from https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc.html
def plot_roc_curve(classifiers, legend, title, X_test, y_test):
    trace1 = go.Scatter(
        x=[0, 1], 
        y=[0, 1], 
        showlegend=False,
        mode="lines",
        name="",
        line = dict(
            color = colors[0],
        ),
    )
    
    data = [trace1]
    aucs = []
    for clf, string, c in zip(classifiers, legend, colors[1:]):
        y_test_roc = np.array([([0, 1] if y else [1, 0]) for y in y_test])
        y_score = clf.predict_proba(X_test)
        
        # Compute ROC curve and ROC area for each class
        fpr = dict()
        tpr = dict()
        roc_auc = dict()
        for i in range(2):
            fpr[i], tpr[i], _ = roc_curve(y_test_roc[:, i], y_score[:, i])
            roc_auc[i] = auc(fpr[i], tpr[i])

        # Compute micro-average ROC curve and ROC area
        fpr["micro"], tpr["micro"], _ = roc_curve(y_test_roc.ravel(), y_score.ravel())
        roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
        aucs.append(roc_auc['micro'])

        trace = go.Scatter(
            x=fpr['micro'], 
            y=tpr['micro'], 
            showlegend=True,
            mode="lines",
            name=string + " (area = %0.2f)" % roc_auc['micro'],
            hoverlabel = dict(
                namelength=30
            ),
            line = dict(
                color = c,
            ),
        )
        data.append(trace)

    layout = go.Layout(
        title=title,
        autosize=False,
        width=550,
        height=550,
        yaxis=dict(
            title='True Positive Rate',
        ),
        xaxis=dict(
            title="False Positive Rate",
        ),
        legend=dict(
            x=0.4,
            y=0.06,
        ),
    )
    fig = go.Figure(data=data, layout=layout)
    return aucs, iplot(fig, filename=title)

In [70]:
classifiers = [gs_lr, gs_knn, gs_svm, gs_dt, gs_rf]
classifier_names = ["Logistic Regression", "KNN", "SVM", "Decision Tree", "Random Forest"]
auc_scores, roc_plot = plot_roc_curve(classifiers, classifier_names, "ROC curve", X_test, y_test)
roc_plot

We can see that the Random Forest classifier has with 0.98 the highest AUC value, followed by the SVM, the KNN and the Decision Tree with 0.96, 0.92 and 0.91, respectively. The Logistic Regression performs worse with an AUC of just 0.81.

Finally, we compute the accuracy, precision, recall and f1-score on the test set for every classifier. The following table shows the results:

In [76]:
accs = []
recalls = []
precision = []
results_table = pd.DataFrame(columns=["accuracy", "precision", "recall", "f1", "auc"])
for (i, clf), name, auc in zip(enumerate(classifiers), classifier_names, auc_scores):
    y_pred = clf.predict(X_test)
    row = []
    row.append(accuracy_score(y_test, y_pred))
    row.append(precision_score(y_test, y_pred))
    row.append(recall_score(y_test, y_pred))
    row.append(f1_score(y_test, y_pred))
    row.append(auc)
    row = ["%.3f" % r for r in row]
    results_table.loc[name] = row

In [77]:
results_table

,accuracy,precision,recall,f1,auc
Logistic Regression,0.760,0.321,0.788,0.456,0.812
KNN,0.859,0.470,0.824,0.598,0.922
SVM,0.901,0.583,0.788,0.670,0.958
Decision Tree,0.889,0.549,0.729,0.626,0.908
Random Forest,0.955,0.831,0.812,0.821,0.978


The results show that the Logistic Regression has with about 0.32 a very low precision value, which means that when it predicts a customer to churn, it is just in 32% of the cases correct.<br/>
Moreover, in accuracy the Decision Tree performs with 0.89 better than the KNN, which achieves 0.86. But in the AUC measure the KNN outperforms with 0.92 the Decision Tree with 0.91.<br/>
The KNN even has the highest recall value, but it achieves poor results for the precision, which also makes its f1-score the second worst of all classifiers.<br/>
The SVM has with 0.58 the second highest precision score but is far behind the best precision score of 0.83 of the Random Forest. This also impacts the f1-score for which the SVM also achieves the second highest with an value of 0.67, but again being worse than the Random Forest, which achieves a f1-score of 0.82.

## Conclusion ##

Our goal was to identify clients which are likely to churn, so we can do special-purpose marketing strategies to avoid the churn event. For this we evaluated differently preprocessed datasets and different classifiers. The analysis has shown that the PCA transformation was not found to be useful. Instead, we suggest to use the whole dataset and apply a oversampling technique in order to deal with the unbalanced target variable. <br/>
In the classification chapter we have trained several different classifiers, including a Logistic Regression, a K-Nearest Neighbors Classifier, a Support-Vector Machine, a Decision Tree and a Random Forest. It was found that the best performance in accuracy, as well as f1-score and AUC is achieved by the Random Forest. One of the most important predictors for the Random Forest is the number of customer service calls. This might imply that the company should improve its customer service. Another important feature is the total day minutes and the total day charge, which basically hold the same information. So the company could try to either lower its charge per minute for clients, which have many day minutes or it could offer flat rates for calls.<br/>
Concluding, we suggest the Telecom company to use the Random Forest model to identify potential churn customers and according to the customers life-time value present them special offers.